# Format DataFrame values

See the [styling docs](https://pandas.pydata.org/pandas-docs/stable/style.html) especially, [this section](https://pandas.pydata.org/pandas-docs/stable/style.html#Finer-Control:-Display-Values).

In [1]:
# Setup

import pandas as pd

## Create toy data

In [2]:
# Create toy data

data = [
    (1000, .624324324),
    (10000, .91241)
]

df = pd.DataFrame.from_records(data, columns=['big_number', 'pct'])

df_exponents = pd.DataFrame.from_dict({
    "col_1": [10 ** i for i in range (10)],
    "col_2": [2 ** i for i in range (10)],
})

## Use `DataFrame.style.format`

By default, Pandas doesn't format numeric values when displaying a DataFrame.

In [3]:
# Show default styling of numbers

df

,big_number,pct
0,1000,0.624324
1,10000,0.912410


However, you can use [`Styler.format()`](https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.format.html) which is accessible as the `.format()` method of the `.style` attribute of a `DataFrame`, i.e. `df.style.format(...)`, to format the values in the `DataFrame`.

### Using different format strings for each column

Most often I specify a dictionary mapping column names to [Python format strings](https://pyformat.info).

In [4]:
# Use style.format

df.style.format({
    'big_number': '{:,}',
    'pct': '{:.1%}',
})

,big_number,pct
0,"1,000",62.4%
1,"10,000",91.2%


### Using a single format string for all values

The above example shows specifying formats with the `formatter` argument by passing a dictionary that maps from column names to format strings.

This argument can also be a single format string, if you want to use the same format for all columns.

In [5]:
# Show unformatted DataFrame

df_exponents

,col_1,col_2
0,1,1
1,10,2
2,100,4
3,1000,8
4,10000,16
5,100000,32
6,1000000,64
7,10000000,128
8,100000000,256
9,1000000000,512


In [6]:
# Format all values using the same format string

df_exponents.style.format("{:,.0f}")

,col_1,col_2
0,1,1
1,10,2
2,100,4
3,"1,000",8
4,"10,000",16
5,"100,000",32
6,"1,000,000",64
7,"10,000,000",128
8,"100,000,000",256
9,"1,000,000,000",512


### Using a function

You can also pass a function to the `Styler.format()`. For example, what if `col_2` represents the number of minutes and I want to format this in `HH:MM:SS` format.

In [7]:
# Show example of formatting using a function

def timedelta_total_hours(td):
    """Returns total hours represented by td, converting days to hours"""
    return (td.components.days * 24) + td.components.hours

def format_timedelta_as_hours(td):
    """Format a timedelta in HH:MM:SS format"""
    total_hours = timedelta_total_hours(td)
    return f"{total_hours:,d}:{td.components.minutes:02d}:{td.components.seconds:02d}"

def format_minutes(minutes):
    """Format number of minutes in HH:MM:SS format"""
    return format_timedelta_as_hours(pd.Timedelta(minutes=minutes))

df_exponents.style.format({
    "col_1": "{:,.0f}",
    "col_2": format_minutes
})

,col_1,col_2
0,1,0:01:00
1,10,0:02:00
2,100,0:04:00
3,"1,000",0:08:00
4,"10,000",0:16:00
5,"100,000",0:32:00
6,"1,000,000",1:04:00
7,"10,000,000",2:08:00
8,"100,000,000",4:16:00
9,"1,000,000,000",8:32:00


## Use the `precision`, and `thousands` arguments

Instead of specifying a format string or function (or dictionary of those) for `formatter`, you can also specify the `precision` and `thousands` arguments to limit the number of decimal points and separate each group of thousands with a comma. There are a few other arguments to `Styler.format()`, so [read the docs](https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.format.html)!

Here's a reminder of what the unformatted data looks like.

In [8]:
# Show the unformatted DataFrame again

df

,big_number,pct
0,1000,0.624324
1,10000,0.912410


In [9]:
# Use the `precision` and `thousands` arguments to `Styler.format()` instead
# of specifying a `formatter`.

df.style.format(precision=2, thousands=",")

,big_number,pct
0,"1,000",0.62
1,"10,000",0.91


## Different formatters for different rows

In the vast majority of cases, I want to vary the format by column, but one case where I would want to vary the format by *rows* is the output of [`Series.describe()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.describe.html).

Consider again the case where we want to treat `col_2` of the `df_exponents` `DataFrame` as minutes.

In [10]:
# Descriptive statistics for `col_2`

df_exponents[["col_2"]].describe()

,col_2
count,10.00000
mean,102.30000
std,164.94851
min,1.00000
25%,5.00000
50%,24.00000
75%,112.00000
max,512.00000


In this case, I might want to format the quartiles, minimum and mean as hours, minutes and seconds, but `count` will always be the number of records, an integer. `Styler.format()` has a `subset` argument, but the way you select rows is completely unintuitive unless you do a lot of advanced indexing.

I found the clearest way to do this is to use [`pd.IndexSlice`](https://pandas.pydata.org/docs/reference/api/pandas.IndexSlice.html) to specify a slice that selects particular rows and columns.

In [11]:
idx = pd.IndexSlice

# Wrap this expression in parens so I can break it up onto multiple lines
# for legibility.
(
    df_exponents[["col_2"]].describe()
        .style
            # Default format for most rows, as hours, minutes and seconds
            .format(format_minutes)
            # Special format for `count`
            .format(
                # Specify the format string
                "{:,.0f}",
                # Specify the rows and columns that will use this formatter
                subset=idx["count", :]
            )
)

,col_2
count,10
mean,1:42:18
std,2:44:56
min,0:01:00
25%,0:05:00
50%,0:24:00
75%,1:52:00
max,8:32:00
